## Import all the relevant Modules 

In [2]:
%matplotlib inline


#TODO: Rename module shortcuts such as hb into something usefull after development
import Modules.M06_Finite_Temperature.Module_Finite_Temperature as ft
from Modules.M00_General.Module_Widgets_and_Sliders import Save_Figure_Button, Click_Save_Figure, set_filename

# default Jupyter widgets
import ipywidgets as widgets
from ipywidgets import HBox, VBox

# Plotting
import seaborn as sns
sns.set_style("darkgrid")
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size':14})

# for printlenght and save figures button
import functools
import numpy as np
np.set_printoptions(linewidth=150) #set output length, default=75

def close_widgets(DIR) -> None:
    """Close all widgets `wi` and displays `di` in notebook directory `dir()`.
    Also clear `Save_Figure_Button`
    """
    for i in range(100):
        if f"w{i}" in DIR:
            exec(f"w{i}.close()")
        if f"d{i}" in DIR:
            exec(f"d{i}.close()")
    # clear `Save_Figure_Button` callbacks, otherwise all previous callbacks are executed and figures are saved multiple times
    Save_Figure_Button._click_handlers.callbacks = []
            

%load_ext autoreload
%autoreload 2

## Finite Temperature
<!---  Define a few convenience macros for bra-ket notation. -->
$\newcommand{\ket}[1]{\left\vert{#1}\right\rangle}$
$\newcommand{\bra}[1]{\left\langle{#1}\right\vert}$
$\newcommand{\braket}[2]{\left\langle{#1}\middle|{#2}\right\rangle}$
$\newcommand{\dyad}[2]{\left|{#1}\middle\rangle\middle\langle{#2}\right|}$
$\newcommand{\mel}[3]{\left\langle{#1}\vphantom{#2#3}\right|{#2}\left|{#3}\vphantom{#1#2}\right\rangle}$
$\newcommand{\expval}[1]{\left\langle{#1}\right\rangle}$
$\newcommand\dif{\mathop{}\!\mathrm{d}}$
$\newcommand\ii{\mathrm{i}}$
$\newcommand{\coloneqq}{\mathop{:=}}$
$\newcommand{\abs}[1]{\left\vert{#1}\right\vert}$
$\newcommand{\vb}[1]{\mathbf{#1}}$
$\newcommand{\im}[1]{\operatorname{Im}{#1}}$
$\newcommand{\re}[1]{\operatorname{Re}{#1}}$

We previously saw how to compute time-dependent correlation functions and electronic propagators (one-particle spectral function), always assuming zero temperature. In this notebook, we will extend our investigation to finite temperature. 

To this end we assume our system which we want to study is weakly coupled to a heat bath at temperature $T$, with which it can exchange energy and/or particles, which would lead to so-called canonical and grand canonical ensembles, respectively. We will focus on the canonical ensemble, which is the most relevant for our purposes, where the system is allowed to exchange energy with the heat bath, but not particles.

Because we now allow for energy exchange we have to modify our view and for a given number of sites $n$, include all possible spin sectors that lead to the same total number of particles $N$. For example, for $n=6$ and $N=6$ we have the following possible spin sectors:

$$
\large
(6, 0), (5, 1), (4, 2), (3, 3), (2, 4), (1, 5), (0, 6),
$$
where the first number in each tuple is the number of spin-up particles and the second number is the number of spin-down particles.

### Ergodic Hypothesis

To compute the averages over finite temperature we will make use of the ergodic hypothesis, which roughly states that the time average of an observable is equal to its ensemble average. This is a very powerful statement, because it allows us to compute the time average of an observable, which is something we can measure in an experiment, by computing the ensemble average, which is something we can compute using our numerical methods. In other words, we assume it is as good to simulate a system over a long time as it is to make many independent realizations of the same system, which is not always true, but often a good approximation.

Under this assumption the thermal average of an observable $\hat{O}$ can be shown to be given by

$$
\large
\expval{\hat{O}}_T = \frac{1}{Z(T)} \sum_{\ell} \exp(-\beta E_{\ell}) \mel{\ell}{\hat{O}}{\ell},
$$
where the inverse temperature $\beta = \frac{1}{k_\mathrm{B} T}$, $k_\mathrm{B}$ is the Boltzmann constant which we will set to 1 for the rest of the derivations, $T$ is the temperature, $E_{\ell}$ is the energy of the $\ell$-th eigenstate of the system, and the exponential factor is the Boltzmann factor which gives the probability of the system to be in the $\ell$-th eigenstate for a given temperature. Finally, $Z(T)$ is the partition function, which is given by the sum of all Boltzmann factors, such that the sum of all probabilities is equal to 1:

$$
\large
Z(T) = \sum_{i} \exp(-\beta E_{i}). 

$$

By inserting the identity operator $\hat{1} = \sum_{n} \dyad{n}{n}$ we can rewrite the thermal average as

$$
\large
\begin{align*}
\expval{\hat{O}}_T &= \frac{1}{Z(T)} \sum_{\ell} \exp(-\beta E_{\ell}) \mel{\ell}{\hat{O}}{\ell} \\
					%
				   &= \frac{1}{Z(T)} \sum_{\ell} \sum_{n} \exp(-\beta E_{\ell}) \mel{\ell}{\hat{O}}{n} \bra{n}\ket{\ell} \\
					%
					&=  \sum_{n} \sum_{\ell} \bra{n}\frac{1}{Z(T)}\ket{\ell} \exp(-\beta E_{\ell}) \mel{\ell}{\hat{O}}{n}  \\
					%
					&=  \sum_{n} \mel{n}{\hat{\rho}\hat{O}}{n}  \\
					%
					&= \mathrm{Tr}(\hat{\rho}\hat{O}),
\end{align*}
$$
where we have defined the density matrix $\hat{\rho}$ as

$$
\large
\hat{\rho} \coloneqq \frac{1}{Z(T)} \sum_{\ell} \exp(-\beta E_{\ell}) \dyad{\ell}{\ell}.
$$

Note that the formula for the thermal average as the trace of the density matrix and the operator is generally true even for mixed systems, while the formula for the density matrix is only true in thermal equilibrium.

In [3]:
#create instance of finite temperature class
h_ft = ft.FiniteTemperature()
h_ft.n.value = 3
h_ft.N.value = 4

# layout of widgets
box_layout = widgets.Layout(border='solid 2px')

### Poor Man's Density of States
The first widget shows all the eigenvalues of the Hubbard Hamiltonian for a system with $n$ sites and $N$ electrons. One can play around with the number of bins and the on-site interaction $U$. Observe how with increasing values of $U$ certain energy clusters form, which are separated by gaps

Note that the update on the widgets takes a few seconds, so please be patient as far more complicated calculations are performed in the background, then in the previous notebooks.

In [ ]:
close_widgets(dir())

#create the widget
w1 = widgets.interactive(h_ft.Plot_Energy_Histogram, u=h_ft.u25, bins=h_ft.bins, n=h_ft.n, N=h_ft.N, box=h_ft.t_ij);

# create the save figure button
filename = set_filename("Energy_Histogram.pdf")
Save_Figure_Button.on_click(functools.partial(Click_Save_Figure, widget=w1, name_widget=filename, output=h_ft.out, path="Figures/"))

# vertical and horizontal box for widgets
d1 = VBox([
    	HBox([Save_Figure_Button, filename, h_ft.out], layout=box_layout),
        HBox([
            VBox(w1.children[0:2], layout=box_layout),
			VBox(w1.children[2:4], layout=box_layout),
			VBox([w1.children[4]], layout=box_layout)
			]),
        w1.children[-1],
		])

display(d1)
w1.update()	# otherwise graph will only appear after the first interaction

### Partition Function
In the next widget once can see the partition function $Z(T)$ as a function of temperature $T$ for a system with $n$ sites and $N$ electrons. One can change the Temperature range and on-site interaction. Observe how the partition function increases with increasing temperature, as expected. Also for certain combinations of $n$ and $N$ the value of the partition function at $T=0$ is not 1, indicating that the ground state is degenerate, which is for example case for $n=6$ and $N=5$.

In [7]:
close_widgets(dir())

#create the widget
w2 = widgets.interactive(h_ft.Plot_Partition_Function_Z, T=h_ft.T_range, u=h_ft.u25, n=h_ft.n, N=h_ft.N, box=h_ft.t_ij);

# create the save figure button
filename = set_filename("Partition_Function_Z.pdf")
Save_Figure_Button.on_click(functools.partial(Click_Save_Figure, widget=w2, name_widget=filename, output=h_ft.out, path="Figures/"))

# vertical and horizontal box for widgets
d2 = VBox([
    	HBox([Save_Figure_Button, filename, h_ft.out], layout=box_layout),
        HBox([
            VBox(w2.children[0:2], layout=box_layout),
			VBox(w2.children[2:4], layout=box_layout),
			VBox([w2.children[4]], layout=box_layout)
			]),
        w2.children[-1],
		])

display(d2)
w2.update()	# otherwise graph will only appear after the first interaction

### Thermodynamic Observables

After deriving the formula for the thermal average and the partition function, we can now derive some thermodynamic observables relevant for our system. We will start with the internal energy $U$, which is just the expectation value of the Hamiltonian given by

$$
\large
U = \expval{\hat{H}}_T = \mathrm{Tr}(\hat{\rho}\hat{H}).
$$

Next we will compute the free energy $F$ which is a useful quantity to compute for further developments. One can derive the free energy from the partition function as

$$
\large
F = -T \ln(Z(T)).
$$

The definition of the free energy allows us to calculate another important quantity, the entropy $S$ of the system via

$$
\large
F = U - T S.
$$

Finally, we will compute the specific heat or heat capacity $C_V$ of the system, which is defined as

$$
\large
c_\mathrm{V} = \frac{\partial U}{\partial T}.
$$

In the widget below all four quantities are plotted as a function of temperature $T$ for a system with $n$ sites and $N$ electrons. One can change the Temperature range and on-site interaction. Observe how the internal energy increases with increasing temperature, while the free energy decreases. The entropy increases with increasing temperature, while the heat capacity shows a peak at a certain temperature.


In [4]:
close_widgets(dir())

#create the widget
w3 = widgets.interactive(h_ft.Plot_Thermodynamic_Observables, T=h_ft.T_range, u=h_ft.u25, n=h_ft.n, N=h_ft.N, box=h_ft.t_ij);

# create the save figure button
filename = set_filename("Thermodynamic_Observables.pdf")
Save_Figure_Button.on_click(functools.partial(Click_Save_Figure, widget=w3, name_widget=filename, output=h_ft.out, path="Figures/"))

# vertical and horizontal box for widgets
d3 = VBox([
    	HBox([Save_Figure_Button, filename, h_ft.out], layout=box_layout),
        HBox([
            VBox(w3.children[0:2], layout=box_layout),
			VBox(w3.children[2:4], layout=box_layout),
			VBox([w3.children[4]], layout=box_layout)
			]),
        w3.children[-1],
		])

display(d3)
w3.update()	# otherwise graph will only appear after the first interaction

We can also ask about the average value of the Spin $S_z$ operator and the square of the spin $S^2$ operator. The average value of the spin operator is given by

$$
\large
\expval{\hat{S}_z} = \mathrm{Tr}(\hat{\rho}\hat{S}_z),
$$
where $\hat{S}_z$ is the spin operator given by

The next to widgets show on the one-hand the matrix elements of both of these operators given by

$$
\large
\mel{\ell}{\hat{S}_z}{\ell} \text{ and } \mel{\ell}{\hat{S}^2}{\ell},
$$
which are both diagonal in the eigenbasis of the Hubbard Hamiltonian. The second widget shows the thermal average of these operators given by

$$
\large
\expval{\hat{S}_z} = \mathrm{Tr}(\hat{\rho}\hat{S}_z) \text{ and } \expval{\hat{S}^2} = \mathrm{Tr}(\hat{\rho}\hat{S}^2),
$$

In [6]:
close_widgets(dir())

#create the widget
w4 = widgets.interactive(h_ft.Plot_Sz_Sz2_T_Elements, n=h_ft.n, N=h_ft.N, box=h_ft.t_ij);

# create the save figure button
filename = set_filename("Sz_Sz2_Matrix_Elements.pdf")
Save_Figure_Button.on_click(functools.partial(Click_Save_Figure, widget=w4, name_widget=filename, output=h_ft.out, path="Figures/"))

# vertical and horizontal box for widgets
d4 = VBox([
    	HBox([Save_Figure_Button, filename, h_ft.out], layout=box_layout),
        HBox([
            VBox(w4.children[0:2], layout=box_layout)
			]),
        w4.children[-1],
		])

display(d4)
w4.update()	# otherwise graph will only appear after the first interaction

In [7]:
close_widgets(dir())

#create the widget
w5 = widgets.interactive(h_ft.Plot_Sz_Sz2_T, T=h_ft.T_range, u=h_ft.u25, n=h_ft.n, N=h_ft.N, box=h_ft.t_ij);

# create the save figure button
filename = set_filename("Thermal_Average_Sz_Sz2.pdf")
Save_Figure_Button.on_click(functools.partial(Click_Save_Figure, widget=w5, name_widget=filename, output=h_ft.out, path="Figures/"))

# vertical and horizontal box for widgets
d5 = VBox([
    	HBox([Save_Figure_Button, filename, h_ft.out], layout=box_layout),
        HBox([
            VBox(w5.children[0:2], layout=box_layout),
			VBox(w5.children[2:4], layout=box_layout),
			VBox([w5.children[4]], layout=box_layout)
			]),
        w5.children[-1],
		])

display(d5)
w5.update()	# otherwise graph will only appear after the first interaction

### Spin-Spin Correlation Function

Finally, we will compute the thermal spin-spin correlation function, which is defined for a given spin operator $\hat{S}_i$ at site $i$ and a given spin operator $\hat{S}_j$ at site $j$ as

$$
\large
\expval{\hat{S}_{i,z}\hat{S}_{j,z}}_T 
$$

In the widget below the thermal spin-spin correlation function is plotted as a function of temperature $T$ for a system with $n$ sites and $N$ electrons. Due to symmetry reasons we only show the relevant different combinations of sites $i$ and $j$. One can change the Temperature range and on-site interaction. Observe how the spin-spin correlation function decreases with increasing temperature.

In [9]:
close_widgets(dir())

#create the widget
w6 = widgets.interactive(h_ft.Plot_SzSz_T, T=h_ft.T_range, u=h_ft.u25, n=h_ft.n, N=h_ft.N, box=h_ft.t_ij);

# create the save figure button
filename = set_filename("Temperature_Spin_Spin_Correlation.pdf")
Save_Figure_Button.on_click(functools.partial(Click_Save_Figure, widget=w6, name_widget=filename, output=h_ft.out, path="Figures/"))

# vertical and horizontal box for widgets
d6 = VBox([
    	HBox([Save_Figure_Button, filename, h_ft.out], layout=box_layout),
        HBox([
            VBox(w6.children[0:2], layout=box_layout),
			VBox(w6.children[2:4], layout=box_layout),
			VBox([w6.children[4]], layout=box_layout)
			]),
        w6.children[-1],
		])

display(d6)
w6.update()	# otherwise graph will only appear after the first interaction